Show git repo organization options?

Show router tree, module import tree, call tree, feature tree?

Warn of changes made outside of the module or feature branch?

Lint all code?  Reformat all code using webstorm?

Code analysis to combine blocks into modules?



TODO: use language server for this

generate unit test from logic branching?


In [6]:
var acorn = require("acorn");
var ASTQ  = require("astq");
var _ = require('lodash');
var importer = require('../Core');

var source = importer.interpret('zuora eloqua mapper').code;
    
var comments = [], tokens = [];

var ast = acorn.parse(source, {
    ecmaVersion: 6,
    // collect ranges for each node
	ranges: true,
	// collect comments in Esprima's format
	onComment: comments,
	// collect token ranges
	onToken: tokens
})

var astq = new ASTQ();
var assignments = astq.query(ast, `
//AssignmentExpression [
    //Identifier[@name=='record']
]
`);
var grouped = _.groupBy(assignments, a => comments.filter(c => c.start < a.start).pop().value.trim());
var output = '';
Object.keys(grouped).forEach(k => {
    const ass = grouped[k].map(as => {
        const leftMembers = as.left.type === 'MemberExpression'
            ? [as.left.property.value] : astq.query(as.left, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        const rightMembers = as.right.type === 'MemberExpression'
            ? [as.right.property.value] : astq.query(as.right, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        if(leftMembers.length === 0 || rightMembers.length === 0) {
            return;
        }
        return `
            it('should map ${leftMembers.join(', ')} to eloqua given ${rightMembers.join(', ')} from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['${rightMembers.join('\'] || records[0][\'')}'], mapped[0]['${leftMembers}']);
            })
`;
    }).filter(m => m).join('\n');
    output += `
        describe('${k} properties mapped', () => {
            ${ass}
        });
`
})
//console.log(astq.query(ast, `//AssignmentExpression`));
var destination = [].concat(...astq.query(ast, `//AssignmentExpression`)
    .map(as => as.left.type === 'MemberExpression'
            ? [as.left.property.value]
            : astq.query(as.left, `//MemberExpression`)
         .map(m => m.property.value)
         )).filter(m => m)
var source = astq.query(ast, `
//MemberExpression [
    //Literal
]
`)
    .filter(e => (e.property.value + '').includes('.'))
    .map(e => e.property.value)
console.log(`
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        ${output}
    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = ${JSON.stringify(source, null, 4).replace(/\n/ig, '\n    ')};
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

`);


reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb

var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        
        describe('contact information properties mapped', () => {
            
            it('should map EmailAddress to eloqua given SoldToContact.WorkEmail, BillToContact.WorkEmail from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['SoldToContact.WorkEmail'] || records[0]['BillToContact.Wo